In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 
headers = {'User Agent':'Mozilla/5.0'}

players = ['paolo-banchero', 'jabari-smith-2', 'jonathan-davis-3', 'ej-liddell', 'julian-champagnie',
           'jamaree-bouyea', 'keon-ellis', 'kennedy-chandler', 'orlando-robinson', 'bryce-mcgowens',
           'blake-wesley', 'christian-koloko', 'kendall-brown', 'chet-holmgren', 'jaden-ivey', 
           'bennedict-mathurin', 'max-christie', 'patrick-baldwinjr', 'tyty-washingtonjr', 'jalen-duren', 
           'malaki-branham', 'jd-notae', 'keegan-murray', 'wendell-moorejr', 'jeremy-sochan', 
           'trevion-williams', 'ochai-agbaji', 'mark-williams-7', 'trayce-jackson-davis', 'jaime-jaquezjr', 
           'johnny-juzang', 'jahvon-quinerly', 'matthew-mayer', 'trevor-keels',  'jaylin-williams-2',
           'walker-kessler', 'kameron-mcgusty', 'michael-devoe',  'iverson-molinar', 'keve-aluma', 
           'christian-braun','tyson-etienne', 'alex-barcello', 'kevin-obanor', 'jalen-williams-13', 
           'brady-manek', 'ryan-rollins', 'dalen-terry', 'ron-harperjr', 'david-roddy', 'andrew-nembhard',
           'izaiah-brockington', 'jd-davison', 'aj-griffin', 'tari-eason','jabari-walker', 'scotty-pippenjr']

player_stats = []
playerlist = []
year_list = []
age_list = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2022')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        playerlist.append(player)
        
    except:
        continue
    try:
        url = f'https://www.google.com/search?q={player}+basketball+birth+date'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        age = (soup.findAll('div', class_ = 'BNeawe iBp4i AP7Wnd')[0])
        for x in age:
            x = x.text
        x = x.replace(',','').split()
        months = {
            'Janurary': '1',
            'February': '2',
            'March': '3',
            'April': '4',
            'May': '5',
            'June': '6',
            'July': '7',
            'August': '8',
            'September': '9',
            'October': '10',
            'November': '11',
            'December': '12'
        } 
        birth_date = months[x[0]]+ '-' + x[1] + '-' + x[2]
        draft_date = '7-1-2022'
        import datetime 
        birth_date_final = datetime.datetime.strptime(birth_date, '%m-%d-%Y')  
        draft_date_final = datetime.datetime.strptime(draft_date, '%m-%d-%Y')

        age_on_draft_night = draft_date_final - birth_date_final
        age = round(age_on_draft_night.days/365, 1)
        age_list.append(age)
        year = round(((1-(age/35))*2.33), 2)
        year_list.append(year)
        
    except:
        age_list.append("No data")
        year_list.append(0)

age_list[1] = 19.1
year_list[1] = 1.07
age_list[2] = 20.3
year_list[2] = .96
age_list[3] = 21.6
year_list[3] = .83
age_list[4] = 21.0 
year_list[4] = .89
age_list[5] = 23.0 
year_list[5] = .69
age_list[6] = 22.5 
year_list[6] = .74
age_list[7] = 19.7 
year_list[7] = 1.01
age_list[8] = 22.0 
year_list[8] = .79
age_list[9] = 19.7 
year_list[9] = 1.01
age_list[10] = 19.3
year_list[10] = 1.05
age_list[11] = 22.4
year_list[11] = .75
age_list[12] = 19.2
year_list[12] = 1.06

table.insert(0, "Name", playerlist)
table.insert(2, "Year", year_list)
table.insert(2, "Age", age_list)

table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

table["Player Grade"] = ((table['PTS']) + (table['TRB']*1.25) + (table['AST']*2) +
(table['BLK']*2) + (table['STL']*3) + (table['3P']*2) + (table['3PA']) + (table['SOS']/2)) * table['Year']

table["Player Grade"] = table["Player Grade"]*1.75
table["Player Grade"] = (round(table["Player Grade"], 1))
table["Player Grade"]= table["Player Grade"].astype(float)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(5, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
del table['Year']
del table['TOV']
del table['PF']
table

,Name,Season,Age,School,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,PTS,,Player Grade
1,Jabari Smith,2021-22,19.1,Auburn,8.17,34,28.8,.429,2.3,5.5,0.420,0.799,7.4,2.0,1.1,1.0,16.9,,92.9
2,Keegan Murray,2021-22,21.9,Iowa,6.87,35,31.9,.554,1.9,4.7,0.398,0.747,8.7,1.5,1.3,1.9,23.5,,86.8
3,Paolo Banchero,2021-22,19.6,Duke,7.26,39,33.0,.478,1.1,3.3,0.338,0.729,7.8,3.2,1.1,0.9,17.2,,85.8
4,Jonathan Davis,2021-22,20.3,Wisconsin,9.16,31,34.2,.427,1.2,3.9,0.306,0.791,8.2,2.1,1.2,0.7,19.7,,84.1
5,Chet Holmgren,2021-22,20.2,Gonzaga,4.50,32,26.9,.607,1.3,3.3,0.390,0.717,9.9,1.9,0.8,3.7,14.1,,83.5
6,Julian Champagnie,2021-22,21.0,St. John's (NY),6.51,31,34.2,.414,2.1,6.2,0.337,0.781,6.6,2.0,2.0,1.1,19.2,,83.0
7,Scotty Pippen Jr,2021-22,21.7,Vanderbilt,8.05,36,33.1,.416,1.8,5.5,0.325,0.749,3.6,4.5,1.9,0.2,20.4,,82.7
8,Bennedict Mathurin,2021-22,20.0,Arizona,6.84,37,32.5,.450,2.2,6.1,0.369,0.764,5.6,2.5,1.0,0.3,17.7,,82.6
9,Kennedy Chandler,2021-22,19.7,Tennessee,9.94,34,30.8,.464,1.4,3.8,0.383,0.606,3.2,4.7,2.2,0.2,13.9,,81.1
10,Jaden Ivey,2021-22,20.4,Purdue,8.23,36,31.4,.460,1.8,5.0,0.358,0.744,4.9,3.1,0.9,0.6,17.3,,78.5
